In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/optiver-trading-at-the-close/public_timeseries_testing_util.py
/kaggle/input/optiver-trading-at-the-close/train.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/sample_submission.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv
/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv
/kaggle/input/optiver-trading-at-the-close/optiver2023/competition.cpython-310-x86_64-linux-gnu.so
/kaggle/input/optiver-trading-at-the-close/optiver2023/__init__.py


In [1]:
import pandas as pd
import numpy as np
import os
import polars as pl
from tqdm import tqdm
import itertools
from itertools import combinations
import warnings
from numba import njit, prange
import time
from functools import wraps
warnings.filterwarnings("ignore")

In [2]:
def time_it(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()  # 记录开始时间
        result = func(*args, **kwargs)  # 调用被装饰的函数
        end_time = time.time()  # 记录结束时间
        execution_time = end_time - start_time  # 计算执行时间
        print(f"Function '{func.__name__}' executed in {execution_time:.4f} seconds")
        return result  # 返回函数结果
    return wrapper

In [3]:
ORIGINAL_TRAIN = pd.read_csv('/kaggle/input/optiver-trading-at-the-close/train.csv',engine = 'pyarrow')

# 1. 市场相关features ——zzc

In [4]:
def market_features(df_1day):
    weights = [
        0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
        0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
        0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
        0.004, 0.004, 0.006, 0.002, 0.002, 0.04, 0.002, 0.002, 0.004, 0.04, 0.002, 0.001,
        0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
        0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
        0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
        0.02, 0.004, 0.006, 0.002, 0.02, 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
        0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
        0.004, 0.006, 0.006, 0.001, 0.04, 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
        0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
        0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
        0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
        0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
        0.04, 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02, 0.004, 0.002, 0.006, 0.02,
        0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
        0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
    ]
    weights = pd.Series(weights)
    weights.index.name = 'stock_id'

    market_feats = pd.DataFrame(
        index=df_1day.index.get_level_values(0).unique())
    wap = df_1day['wap'].unstack()
    market_feats['index_wap_std'] = wap.std(axis=1)
    stock_ret1 = 10000*(wap/wap.shift(1)-1)
    market_feats['index_ret1'] = stock_ret1.mean(axis=1)
    market_feats['index_ret1_std'] = stock_ret1.std(axis=1)
    market_feats['index_ret1_w'] = stock_ret1.mul(weights, axis=1).sum(axis=1)
    stock_ret6 = 10000*(wap/wap.shift(6)-1)
    market_feats['index_ret6'] = stock_ret6.mean(axis=1)
    market_feats['index_ret6_std'] = stock_ret6.std(axis=1)
    market_feats['index_ret6_w'] = stock_ret6.mul(weights, axis=1).sum(axis=1)
    ask_size = df_1day['ask_size'].unstack()
    bid_size = df_1day['bid_size'].unstack()
    market_feats['market_volume'] = ask_size.mean(axis=1)+bid_size.mean(axis=1)
    ask_price = df_1day['ask_price'].unstack()
    bid_price = df_1day['bid_price'].unstack()
    market_feats['market_spread'] = ask_price.mean(
        axis=1)-bid_price.mean(axis=1)
    market_feats['market_max_spread'] = ask_price.max(
        axis=1)-bid_price.min(axis=1)
    market_feats['mfd'] = market_feats.index // 60
    
    return market_feats
@time_it
def cal_market_features(df):
    feature_0 = df.set_index(['seconds_in_bucket', 'stock_id'])
    feature_market = feature_0.groupby('date_id').apply(market_features)
    feature_market = feature_0.reset_index()[['stock_id', 'date_id', 'seconds_in_bucket']].merge(right=feature_market, on=['date_id','seconds_in_bucket'], how='left')
    #feature_market.insert(2, 'seconds_in_bucket', feature_market.pop('seconds_in_bucket'))

    return feature_market
market_features = cal_market_features(ORIGINAL_TRAIN.copy())
market_features = pl.DataFrame(market_features)

Function 'cal_market_features' executed in 21.4622 seconds


In [8]:
market_features.write_parquet("market_features.parquet")

# 2. 截面因子——zzh

In [9]:
@time_it
def cross_section_features_from_word(df):
    # 第一批特征：2个价格的差、乘、imblance，3个价格的最大、最小、中位数、imbalance
    prices = ['reference_price','far_price', 'near_price', 'ask_price', 'bid_price', 'wap']
    for c in itertools.combinations(prices, 2):
        df[f'{c[0]}_minus_{c[1]}'] = (df[f'{c[0]}'] - df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_times_{c[1]}'] = (df[f'{c[0]}'] * df[f'{c[1]}']).astype(np.float32)
        df[f'{c[0]}_{c[1]}_imb'] = df.eval(f'({c[0]}-{c[1]})/({c[0]}+{c[1]})')

    for c in itertools.combinations(prices, 3):
        max_ = df[list(c)].max(axis=1)
        min_ = df[list(c)].min(axis=1)
        mid_ = df[list(c)].sum(axis=1)- min_ - max_
        df[f'{c[0]}_{c[1]}_{c[2]}_imb2'] = (max_ - mid_) / (mid_ - min_)

    df['mid_price'] = ((df['ask_price'] + df['bid_price']) / 2).astype(np.float32)
    df['wap_2'] = ((df['bid_price'] * df['bid_size'] + df['ask_price'] * df['ask_size']) / (df['ask_price'] + df['bid_price'])).astype(np.float32)
    df['best_bid_size'] = (df['bid_price'] * df['bid_size']).astype(np.float32)
    df['best_ask_size'] = (df['ask_price'] * df['ask_size']).astype(np.float32)
    df['bid_price_over_ask_price'] = (df['bid_price'] / df['ask_price']).astype(np.float32)
    df['best_bid_and_ask_size'] = (df['best_bid_size'] + df['best_ask_size']) / 2
    df['BBO_size_imbalance'] = (df['best_bid_size'] - df['best_ask_size']) / (df['best_bid_size'] + df['best_ask_size'])
    df['effective_spread'] = 2 * np.abs(df['wap'] - 0.5 * (df.ask_price + df.bid_price)) / (0.5 * (df.ask_price + df.bid_price))
    df['effective_spread'] = df['effective_spread'].astype(np.float32)
    df['market_depth_vol'] = (df.bid_size + df.ask_size).astype(np.float32)
    df['ask_bid_size_diff'] = (df.ask_size - df.bid_size).astype(np.float32)
    df['bid_size_over_ask_size'] = (df.bid_size / df.ask_size).astype(np.float32)
    df['shallowLIX'] = np.log10(df.matched_size / (df.ask_size + df.bid_size)).astype(np.float32)
    df['imbalance_ratio'] = (df.imbalance_size / (df.matched_size + df.imbalance_size)).astype(np.float32)
    df['imb_s1'] = ((df.bid_size - df.ask_size) / (df.bid_size + df.ask_size)).astype(np.float32)
    df['imb_s2'] = ((df.imbalance_size - df.matched_size) / (df.imbalance_size + df.matched_size)).astype(np.float32)
    median_vol = df.groupby("stock_id")['bid_size'].median() + df.groupby("stock_id")['ask_size'].median()
    df['median_vol'] = df['stock_id'].map(median_vol.to_dict()).astype(np.float32)
    df['high_volume'] = np.where(df['bid_price'] + df['ask_price'] > df['median_vol'], 1, 0).astype(np.float32)
    df['high_volume_imbalance_size'] = (df['high_volume'] * df['imbalance_size']).astype(np.float32)
    df['low_volume_matched_size'] = (df['high_volume'] * df['matched_size']).astype(np.float32)
    # 一些关于volume的特征
    max_vol = df.groupby("stock_id")[['bid_size', 'ask_size']].max().sum(axis=1)
    df['max_vol'] = df['stock_id'].map(max_vol.to_dict()).astype(np.float32)
    min_vol = df.groupby("stock_id")[['bid_size', 'ask_size']].min().sum(axis=1)
    df['min_vol'] = df['stock_id'].map(min_vol.to_dict()).astype(np.float32)
    std_vol = df.groupby("stock_id")[['bid_size', 'ask_size']].std().sum(axis=1)
    df['std_vol'] = df['stock_id'].map(std_vol.to_dict()).astype(np.float32)

    first_5min_volume = df.loc[(df.far_price.isna()) | (df.near_price.isna())].groupby('stock_id')[['bid_size', 'ask_size']].median().sum(axis=1)
    df['first_5min_volume'] = df['stock_id'].map(first_5min_volume.to_dict()).astype(np.float32)
    last_5min_volume = df[["stock_id",'bid_size','ask_size','near_price','far_price']].dropna().groupby('stock_id')[['bid_size','ask_size']].median().sum(axis=1)
    df['last_5min_volume'] = df['stock_id'].map(last_5min_volume.to_dict()).astype(np.float32)
    df['first_5min'] = np.where((df.far_price.isna()) | (df.near_price.isna()), 1, 0).astype(np.float32)
    df['last_5min'] = np.where((~df.far_price.isna()) & (~df.near_price.isna()) & (~df.stock_id.isna()) & (~df.bid_size.isna()) & (~df.ask_size.isna()), 1, 0).astype(np.float32)
    df['LogquoteSlope'] = ((np.log10(df.ask_price) - np.log10(df.bid_price)) / (np.log10(df.ask_price) + np.log10(df.bid_price))).astype(np.float32)
    middle_price = (df.ask_price + df.bid_price) / 2
    vwap_a = df.groupby("stock_id")[['ask_price', 'ask_size']].apply(lambda x: (x['ask_price'] * x['ask_size']).sum()) / df.groupby("stock_id")['ask_size'].sum()
    dolvol_a = df['stock_id'].map(df.groupby("stock_id")[['ask_price', 'ask_size']].apply(lambda x: (x['ask_price'] * x['ask_size']).sum()).to_dict()).astype(np.float32)
    df['vwap_a'] = df['stock_id'].map(vwap_a.to_dict()).astype(np.float32)
    vwap_b = df.groupby("stock_id")[['bid_price', 'bid_size']].apply(lambda x: (x['bid_price'] * x['bid_size']).sum()) / df.groupby("stock_id")['bid_size'].sum()
    dolvol_b = df['stock_id'].map(df.groupby("stock_id")[['bid_price', 'bid_size']].apply(lambda x: (x['bid_price'] * x['bid_size']).sum()).to_dict()).astype(np.float32)
    df['vwap_b'] = df['stock_id'].map(vwap_b.to_dict()).astype(np.float32)
    df['vwapm_a'] = ((df['vwap_a'] - middle_price) / middle_price).astype(np.float32)
    df['vwapm_b'] = ((df['vwap_b'] - middle_price) / middle_price).astype(np.float32)
    df['MCI_a'] = (df['vwapm_a'] / dolvol_a).astype(np.float32)
    df['MCI_b'] = (df['vwapm_b'] / dolvol_b).astype(np.float32)
    # imbalance_buy_sell_flag的one-hot特征
    df['imbalance_buy_flag'] = df['imbalance_buy_sell_flag'].where(df['imbalance_buy_sell_flag']==1,0).astype(np.float32)
    df['imbalance_sell_flag'] = -df['imbalance_buy_sell_flag'].where(df['imbalance_buy_sell_flag']==-1,0).astype(np.float32)
    last_5_min_match_size_sum = df.loc[df.seconds_in_bucket >= 300].groupby(["stock_id", "date_id"])['matched_size'].sum()
    all_match_size_sum = df.groupby(["stock_id", "date_id"])['matched_size'].sum()
    ratio = last_5_min_match_size_sum / all_match_size_sum
    ratio
    # 集合竞价成交占比因子：最后五分钟的match_size的和与全部十分钟的match_size的和的比值在过去N天内的平均
    last_5_min_match_size_sum = df.loc[df.seconds_in_bucket >= 300].groupby(["stock_id", "date_id"])['matched_size'].sum()
    all_match_size_sum = df.groupby(["stock_id", "date_id"])['matched_size'].sum()
    ratio = last_5_min_match_size_sum / all_match_size_sum
    for n in tqdm(range(3, 10, 2)):
        factor = ratio.groupby(level=0).transform(lambda x: x.rolling(n, min_periods=1).mean())
        factor.name = f"CMV_{n}"
        factor = factor.reset_index(drop=False)
        df = pd.merge(left=df, right=factor, on=['stock_id', 'date_id'], how='left')
    # 尾盘信息因子：利用bid_size, ask_size, market_depth_vol, best_bid_size, best_ask_size做同样的逻辑处理
    features = ["bid_size", "ask_size", "market_depth_vol", "best_bid_size", "best_ask_size"]
    for feature in tqdm(features):
        last_5_min_size_sum = df.loc[df.seconds_in_bucket >= 300].groupby(["stock_id", "date_id"])[feature].sum()
        all_size_sum = df.groupby(["stock_id", "date_id"])[feature].sum()
        ratio = last_5_min_size_sum / all_size_sum
        for n in range(3, 10, 2):
            factor = ratio.groupby(level=0).transform(lambda x: x.rolling(n, min_periods=1).mean())
            factor.name = f"CMV_{n}_{feature}"
            factor = factor.reset_index(drop=False)
            df = pd.merge(left=df, right=factor, on=['stock_id', 'date_id'], how='left')
    relative_to_max = df.groupby(["stock_id", "date_id"])['reference_price'].apply(lambda x: x.tail(1).values[0] / x.max() - 1).rename("relative_to_max").reset_index(drop=False)
    df = pd.merge(left=df, right=relative_to_max, on=['stock_id', 'date_id'], how='left')
    relative_to_min = df.groupby(["stock_id", "date_id"])['reference_price'].apply(lambda x: x.tail(1).values[0] / x.min() - 1).rename("relative_to_min").reset_index(drop=False)
    df = pd.merge(left=df, right=relative_to_min, on=['stock_id', 'date_id'], how='left')
    # ZXY: 和第一部分重复了
    # 流动性
    df['Liquidity_1'] = (df.far_price / df.wap - 1) / df.matched_size
    df['Liquidity_2'] = (df.far_price / df.mid_price - 1) / df.matched_size
    df['Liquidity_3'] = (df.far_price / df.near_price - 1) / df.matched_size
    df['Liquidity_4'] = (df.far_price / df.reference_price - 1) / df.matched_size
    df['Liquidity_5'] = (df[["reference_price", "far_price", "near_price", "wap"]].max(axis=1) / df[["reference_price", "far_price", "near_price", "wap"]].min(axis=1) - 1) / df.matched_size
    # 收益趋势相关
    # 各个价格和时间的相关性
    for feature in ['mid_price', 'far_price', 'near_price', 'reference_price']:
        factor = df.groupby(["stock_id", "date_id"])[[feature, 'seconds_in_bucket']].apply(lambda x: x[feature].corr(x['seconds_in_bucket']))
        factor.name = f"{feature}_corr"
        df = pd.merge(left=df, right=factor.reset_index(drop=False), on=['stock_id', 'date_id'], how='left').fillna(0.0)

    # cumprod(FarPrice/(BidPrice_(t) + AskPrice_(t )))
    tmp = df[['seconds_in_bucket', 'stock_id', "date_id"]].copy()
    tmp['far_mid_val'] = df['far_price'] / df['mid_price']
    tmp = tmp.set_index(["stock_id", "date_id", "seconds_in_bucket"]).groupby(level=[0, 1]).cumprod()
    df = df.merge(tmp.reset_index(drop=False), on=["stock_id", "date_id", "seconds_in_bucket"], how='left')
    # cumprod(wap_t/wap_(t-1))
    wap_trend = df[["stock_id", "date_id", "seconds_in_bucket", "wap"]].copy().set_index(["stock_id", "date_id", "seconds_in_bucket"])
    wap_trend = wap_trend.groupby(level=[0, 1]).pct_change(fill_method=None)
    wap_trend = (wap_trend + 1).groupby(level=[0, 1]).cumprod()
    df = pd.merge(left=df, right=wap_trend.reset_index(drop=False), on=['stock_id', 'date_id', 'seconds_in_bucket'], how='left')
    mid_trend = df[["stock_id", "date_id", "seconds_in_bucket", "mid_price"]].copy().set_index(["stock_id", "date_id", "seconds_in_bucket"])
    mid_trend = mid_trend.groupby(level=[0, 1]).pct_change(fill_method=None)
    mid_trend = (mid_trend + 1).groupby(level=[0, 1]).cumprod()
    df = pd.merge(left=df, right=mid_trend.reset_index(drop=False), on=['stock_id', 'date_id', 'seconds_in_bucket'], how='left')
    return df
cross_section_features = cross_section_features_from_word(ORIGINAL_TRAIN.copy())


100%|██████████| 5/5 [01:32<00:00, 18.55s/it]


Function 'cross_section_features_from_word' executed in 508.0158 seconds


In [10]:
cross_section_features = pl.DataFrame(cross_section_features)
cross_section_features.write_parquet("cross_section_features.parquet")

# 3. 时序因子——djh

In [11]:
@time_it
def timeseris_features_from_word(df):
    df['matched_percent'] = df['matched_size'] / (df['matched_size'] + df['imbalance_size'])

    # 个股买卖强弱（时序）
    is_buy = df['imbalance_buy_sell_flag'] == 1
    is_sell = df['imbalance_buy_sell_flag'] == -1
    is_flat = df['imbalance_buy_sell_flag'] == 0

    # Using rolling windows and summing the boolean values directly
    df['buy_count'] = is_buy.rolling(window=11).sum().shift(-10)
    df['sell_count'] = is_sell.rolling(window=11).sum().shift(-10)
    df['flat_count'] = is_flat.rolling(window=11).sum().shift(-10)
    df['buy_ratio'] = df['buy_count'] / 11
    df['sell_ratio'] = df['sell_count'] / 11
    df['flat_ratio'] = df['flat_count'] / 11

    # 不平衡方向的 imbalance_size
    df['buy_sell_imbalance_size'] = df['imbalance_size'] * df['imbalance_buy_sell_flag']

    # flag 方向发生变化的时间点
    df['imb_change'] = df['buy_sell_imbalance_size'].diff()
    unique_stock_count = (df.groupby(['date_id', 'seconds_in_bucket', 'imbalance_buy_sell_flag'])['stock_id']
                      .nunique()
                      .unstack(fill_value=0)
                      .reset_index()
                      .rename(columns={1: 'unique_buy_stock_count', 
                                       0: 'unique_flat_stock_count', 
                                      -1: 'unique_sell_stock_count'}))

    df = df.merge(unique_stock_count, on=['date_id', 'seconds_in_bucket'], how='left')

    # 添加买卖强弱与市场作比较（横截面排序）
    df['buy_imb_degree_rank'] = df['buy_count'].rank()
    df['sell_imb_degree_rank'] = df['sell_count'].rank()
    df['flat_imb_degree_rank'] = df['flat_count'].rank()
    # 不同时间段统计特征构建的因子
    df['time_bucket'] = df['seconds_in_bucket'] // 110

    aggregations = {
        'reference_price': ['mean', 'std', 'median'],
        'bid_price': ['mean', 'std', 'median'],
        'ask_price': ['mean', 'std', 'median'],
        'imbalance_size': ['mean', 'std', 'median'],
        'matched_size': ['mean', 'std', 'median'],
        'ask_size': ['mean', 'std', 'median'],
        'bid_size': ['mean', 'std', 'median']
    }

    # Compute aggregations directly for mean, std, and median
    agg_df = df.groupby(['stock_id', 'date_id', 'time_bucket']).agg(aggregations).reset_index()

    # Flatten MultiIndex columns
    agg_df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in agg_df.columns]

    # Custom functions for skew and kurtosis (applied selectively on required columns)
    for col in ['reference_price', 'bid_price', 'ask_price', 'imbalance_size', 'matched_size', 'ask_size', 'bid_size']:
        agg_df[f'skew_{col}'] = df.groupby(['stock_id', 'date_id', 'time_bucket'])[col].skew().values
    #     agg_df[f'kurt_{col}'] = df.groupby(['stock_id', 'date_id', 'time_bucket'])[col].kurt().values
    matched_size_quantiles = (
        df.groupby(['date_id'])['matched_size']
        .quantile([0.5, 0.75, 0.90])
        .unstack(level=-1)
        .rename(columns={0.5: 'median_matched_size', 0.75: 'q75_matched_size', 0.90: 'q90_matched_size'})
        .reset_index()
    )

    # 将分位数与原始数据合并
    df = df.merge(matched_size_quantiles, on='date_id', how='left')

    # 根据 matched_size 对订单大小分类
    conditions = [
        (df['matched_size'] <= df['median_matched_size']),
        (df['matched_size'] > df['median_matched_size']) & (df['matched_size'] <= df['q75_matched_size']),
        (df['matched_size'] > df['q75_matched_size'])
    ]
    choices = ['small', 'medium', 'large']
    df['order_size_category'] = np.select(conditions, choices)

    # 计算分组统计特征
    order_stats = (
        df.groupby(['order_size_category', 'date_id'])
        .agg(
            os_order_count=('matched_size', 'count'),
            os_mean_reference_price=('reference_price', 'mean'),
            os_std_reference_price=('reference_price', 'std'),
            os_median_reference_price=('reference_price', 'median'),
            os_skew_reference_price=('reference_price', 'skew'),

            os_mean_bid_price=('bid_price', 'mean'),
            os_std_bid_price=('bid_price', 'std'),
            os_median_bid_price=('bid_price', 'median'),
            os_skew_bid_price=('bid_price', 'skew'),

            os_mean_ask_price=('ask_price', 'mean'),
            os_std_ask_price=('ask_price', 'std'),
            os_median_ask_price=('ask_price', 'median'),
            os_skew_ask_price=('ask_price','skew')
        )
    ).reset_index()

    # 将统计特征与原始数据合并
    df = df.merge(order_stats, on=['order_size_category', 'date_id'], how='left')
    # 计算bid_size和ask_size的滚动标准差
    df['bid_size_std_30_rolling'] = df['bid_size'].rolling(window=30, min_periods=1).std()
    df['bid_size_std_10_rolling'] = df['bid_size'].rolling(window=10, min_periods=1).std()
    df['bid_size_std_5_rolling'] = df['bid_size'].rolling(window=5, min_periods=1).std()

    df['ask_size_std_30_rolling'] = df['ask_size'].rolling(window=30, min_periods=1).std()
    df['ask_size_std_10_rolling'] = df['ask_size'].rolling(window=10, min_periods=1).std()
    df['ask_size_std_5_rolling'] = df['ask_size'].rolling(window=5, min_periods=1).std()

    # 计算reference_price的收益率
    df['reference_price_ret'] = df['reference_price'].pct_change(fill_method=None)
    df['matched_size_ret'] = df['matched_size'].pct_change(fill_method=None)
    df['far_price_ret'] = df['far_price'].pct_change(fill_method=None)
    df['near_price_ret'] = df['near_price'].pct_change(fill_method=None)
    df['bid_price_ret'] = df['bid_price'].pct_change(fill_method=None)
    df['ask_price_ret'] = df['ask_price'].pct_change(fill_method=None)
    df['wap_ret'] = df['wap'].pct_change(fill_method=None)

    # 计算前一分钟收益率的标准差和偏度
    df['reference_price_ret_std_60'] = df.groupby('stock_id')['reference_price'].rolling(window=6, min_periods=1).std().reset_index(level=0, drop=True)
    df['reference_price_skew_60'] = df.groupby('stock_id')['reference_price'].rolling(window=6, min_periods=1).skew().reset_index(level=0, drop=True)

    # 将收益分成涨跌
    df['gain'] = df['wap_ret'].apply(lambda x: x if x > 0 else 0)
    df['loss'] = df['wap_ret'].apply(lambda x: -x if x < 0 else 0)
    # 计算 RSI（14日周期）
    df['avg_gain'] = df.groupby('stock_id')['gain'].rolling(window=11, min_periods=1).mean().reset_index(level=0, drop=True)
    df['avg_loss'] = df.groupby('stock_id')['loss'].rolling(window=11, min_periods=1).mean().reset_index(level=0, drop=True)
    df['rs'] = df['avg_gain'] / df['avg_loss']
    df['rsi_14'] = 100 - (100 / (1 + df['rs']))
    
    # 计算 reference_price 和 matched_size 的相关系数
    prices = ['reference_price', 'bid_price', 'ask_price', 'wap', 'far_price','near_price']
    sizes = ['matched_size', 'imbalance_size', 'ask_size','bid_size']
    for price in prices:
        for size in sizes:

            if price in df.columns and size in df.columns:
                # 计算滚动相关性，并赋值给新列
                df[f'{price}_{size}_corr_60'] = df.groupby('stock_id').apply(lambda x: x[price].rolling(window=6, min_periods=1).corr(x[size]),include_groups=False).reset_index(level=0, drop=True)
                # 计算1分钟和2分钟的滚动总和
                rolling_size_sum_1min = df[size].rolling(window=5, min_periods=1).sum()
                rolling_size_sum_2min = df[size].rolling(window=11, min_periods=1).sum()

                # 计算 VWAP 1分钟和2分钟
                price_sum_1min = df[price].rolling(window=5, min_periods=1).sum()
                price_sum_2min = df[price].rolling(window=11, min_periods=1).sum()

                df[f'{price}_vwap_1min'] = price_sum_1min / rolling_size_sum_1min
                df[f'{price}_vwap_2min'] = price_sum_2min / rolling_size_sum_2min

                # 计算等权均值 1 分钟和 2 分钟
                df[f'{price}_simple_mean_1min'] = df[price].rolling(window=5, min_periods=1).mean()
                df[f'{price}_simple_mean_2min'] = df[price].rolling(window=11, min_periods=1).mean()

                # 计算 VWAP 和等权均值的差值
                df[f'{price}_vwap_minus_simple_1min'] = df[f'{price}_vwap_1min'] - df[f'{price}_simple_mean_1min']
                df[f'{price}_vwap_minus_simple_2min'] = df[f'{price}_vwap_2min'] - df[f'{price}_simple_mean_2min']
                
    # 计算 bid_size 和 ask_size 的相关系数 rho
    window_size = 5
    df['rho'] = df[['bid_size', 'ask_size']].rolling(window=window_size).corr().unstack().iloc[:,1]

    # 计算 best ask size 和 best bid size 之差
    df['size_diff'] = df['ask_size'] - df['bid_size']
    df['size_sum'] = df['ask_size'] + df['bid_size']

    # 根据公式计算 p_up
    df['p_up'] = np.arctan(np.sqrt((1 + df['rho']) / (1 - df['rho'])) * df['size_diff'] / df['size_sum']) / (2 * np.arctan(np.sqrt((1 + df['rho']) / (1 - df['rho']))))

    # 计算 CL_up 和 CL_down
    window_size_bolling = 11
    window_size_cmo = 19

    for price in prices:
        # 计算布林带的滚动均值和标准差
        rolling_mean = df[price].rolling(window=window_size_bolling).mean()
        rolling_std = df[price].rolling(window=window_size_bolling).std()

        # 布林带的上下限和中线
        df[f'{price}_BB_upper'] = rolling_mean + 2 * rolling_std
        df[f'{price}_BB_lower'] = rolling_mean - 2 * rolling_std
        df[f'{price}_BB_middle'] = rolling_mean

        # 计算价格的上涨和下跌信号
        df[f'{price}_CL_up'] = (df[price] > df[price].shift(window_size_cmo)).astype(float)
        df[f'{price}_CL_down'] = (df[price] < df[price].shift(window_size_cmo)).astype(float)

        # 计算 S_u 和 S_d 的累加
        df[f'{price}_S_u'] = (df[price] * df[f'{price}_CL_up']).rolling(window=window_size_cmo).sum()
        df[f'{price}_S_d'] = (df[price] * df[f'{price}_CL_down']).rolling(window=window_size_cmo).sum()

        # 计算 CMO 指标
        df[f'{price}_CMO'] = 100 * (df[f'{price}_S_u'] - df[f'{price}_S_d']) / (df[f'{price}_S_u'] + df[f'{price}_S_d'])

    return df
timeseries_features = timeseris_features_from_word(ORIGINAL_TRAIN.copy())

Function 'timeseris_features_from_word' executed in 380.7489 seconds


In [12]:
timeseries_features = pl.DataFrame(timeseries_features)

In [13]:
timeseries_features.write_parquet("timeseries_features.parquet")

# 3. 来自T9及图片中的因子——zwj

In [8]:
@time_it
def rank9_features_from_pic(df):
    df_pl = pl.DataFrame(df)
    # -------------pic--------------
    base_features = [ 'imbalance_size','reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap']
    df_pl = df_pl.with_columns(pl.when(pl.col('seconds_in_bucket')<300).then(0).when(pl.col('seconds_in_bucket')<480).then(1).otherwise(2).cast(pl.Float32).alias("seconds_in_bucket_group"))
    df_pl = df_pl.with_columns([
        (pl.col(col).first() / pl.col(col))
        .over(['date_id', 'seconds_in_bucket_group', 'stock_id'])
        .cast(pl.Float32)
        .alias(f"{col}_group_first_ratio")
        for col in base_features
    ])
    df_pl = df_pl.with_columns([
        (pl.col(col).rolling_mean(window_size=100, min_periods=1) / pl.col(col))
        .over(['date_id', 'seconds_in_bucket_group', 'stock_id'])
        .cast(pl.Float32)
        .alias(f"{col}_group_expanding_mean_108")
        for col in base_features
    ])
    df_pl = df_pl.with_columns([
        # 计算每列的分组平均值比率
        (pl.col(col).mean() / pl.col(col))
        .over(['date_id', 'seconds_in_bucket'])
        .cast(pl.Float32)
        .alias(f"{col}_seconds_in_bucket_group_mean_ratio")
        for col in base_features
    ] + [
        # 计算每列的分组内排名比例
        (pl.col(col).rank(descending=True, method='ordinal') / pl.col(col).count())
        .over(['date_id', 'seconds_in_bucket'])
        .cast(pl.Float32)
        .alias(f"{col}_seconds_in_bucket_group_rank")
        for col in base_features
    ])
    return df_pl
    # -------------rank9--------------
    """
    weights = [
    0.004, 0.001, 0.002, 0.006, 0.004, 0.004, 0.002, 0.006, 0.006, 0.002, 0.002, 0.008,
    0.006, 0.002, 0.008, 0.006, 0.002, 0.006, 0.004, 0.002, 0.004, 0.001, 0.006, 0.004,
    0.002, 0.002, 0.004, 0.002, 0.004, 0.004, 0.001, 0.001, 0.002, 0.002, 0.006, 0.004,
    0.004, 0.004, 0.006, 0.002, 0.002, 0.04 , 0.002, 0.002, 0.004, 0.04 , 0.002, 0.001,
    0.006, 0.004, 0.004, 0.006, 0.001, 0.004, 0.004, 0.002, 0.006, 0.004, 0.006, 0.004,
    0.006, 0.004, 0.002, 0.001, 0.002, 0.004, 0.002, 0.008, 0.004, 0.004, 0.002, 0.004,
    0.006, 0.002, 0.004, 0.004, 0.002, 0.004, 0.004, 0.004, 0.001, 0.002, 0.002, 0.008,
    0.02 , 0.004, 0.006, 0.002, 0.02 , 0.002, 0.002, 0.006, 0.004, 0.002, 0.001, 0.02,
    0.006, 0.001, 0.002, 0.004, 0.001, 0.002, 0.006, 0.006, 0.004, 0.006, 0.001, 0.002,
    0.004, 0.006, 0.006, 0.001, 0.04 , 0.006, 0.002, 0.004, 0.002, 0.002, 0.006, 0.002,
    0.002, 0.004, 0.006, 0.006, 0.002, 0.002, 0.008, 0.006, 0.004, 0.002, 0.006, 0.002,
    0.004, 0.006, 0.002, 0.004, 0.001, 0.004, 0.002, 0.004, 0.008, 0.006, 0.008, 0.002,
    0.004, 0.002, 0.001, 0.004, 0.004, 0.004, 0.006, 0.008, 0.004, 0.001, 0.001, 0.002,
    0.006, 0.004, 0.001, 0.002, 0.006, 0.004, 0.006, 0.008, 0.002, 0.002, 0.004, 0.002,
    0.04 , 0.002, 0.002, 0.004, 0.002, 0.002, 0.006, 0.02 , 0.004, 0.002, 0.006, 0.02,
    0.001, 0.002, 0.006, 0.004, 0.006, 0.004, 0.004, 0.004, 0.004, 0.002, 0.004, 0.04,
    0.002, 0.008, 0.002, 0.004, 0.001, 0.004, 0.006, 0.004,
    ]
    weights = {int(k):v for k,v in enumerate(weights)}
    df['weight']  = df['stock_id'].map(weights)
    size_col = ['imbalance_size','matched_size','bid_size','ask_size']
    for _ in size_col:
        df[f'scale_{_}'] = df[_] / df.groupby(['stock_id'])[_].median()
    # auc_ask_size、auc_ask_size，买卖单amount 
    df['auc_bid_size'] = df['matched_size']
    df['auc_ask_size'] = df['matched_size']
    df.loc[df['imbalance_buy_sell_flag']==1,'auc_bid_size'] += df.loc[df['imbalance_buy_sell_flag']==1,'imbalance_size']
    df.loc[df['imbalance_buy_sell_flag']==-1,'auc_ask_size'] += df.loc[df['imbalance_buy_sell_flag']==-1,'imbalance_size']
    df = pl.from_pandas(df)
    feas_list = ['stock_id','seconds_in_bucket','imbalance_size','imbalance_buy_sell_flag',
                'reference_price','matched_size','far_price','near_price','bid_price','bid_size',
                'ask_price','ask_size','wap','scale_imbalance_size','scale_matched_size','scale_bid_size','scale_ask_size'
                    ,'auc_bid_size','auc_ask_size']
    # 基础特征
    df = df.with_columns([
        # 阶段1
        (pl.col('ask_size') * pl.col('ask_price')).alias("ask_money"),
        (pl.col('bid_size') * pl.col('bid_price')).alias("bid_money"),
        (pl.col('ask_size') + pl.col("auc_ask_size")).alias("ask_size_all"),
        (pl.col('bid_size') + pl.col("auc_bid_size")).alias("bid_size_all"),
        (pl.col('ask_size') + pl.col("auc_ask_size") + pl.col('bid_size') + pl.col("auc_bid_size")).alias("volumn_size_all"),
        (pl.col('reference_price') * pl.col('auc_ask_size')).alias("ask_auc_money"),
        (pl.col('reference_price') * pl.col('auc_bid_size')).alias("bid_auc_money"),
        (pl.col('ask_size') * pl.col('ask_price') + pl.col('bid_size') * pl.col('bid_price')).alias("volumn_money"),
        (pl.col('ask_size') + pl.col('bid_size')).alias('volume_cont'),
        (pl.col('ask_size') - pl.col('bid_size')).alias('diff_ask_bid_size'),
        (pl.col('imbalance_size') + 2 * pl.col('matched_size')).alias('volumn_auc'),
        ((pl.col('imbalance_size') + 2 * pl.col('matched_size')) * pl.col("reference_price")).alias('volumn_auc_money'),
        ((pl.col('ask_price') + pl.col('bid_price'))/2).alias('mid_price'),
        ((pl.col('near_price') + pl.col('far_price'))/2).alias('mid_price_near_far'),
        (pl.col('ask_price') - pl.col('bid_price')).alias('price_diff_ask_bid'),
        (pl.col('ask_price') / pl.col('bid_price')).alias('price_div_ask_bid'),
        (pl.col('imbalance_buy_sell_flag') * pl.col('scale_imbalance_size')).alias('flag_scale_imbalance_size'),
        (pl.col('imbalance_buy_sell_flag') * pl.col('imbalance_size')).alias('flag_imbalance_size'),
        (pl.col('imbalance_size') / pl.col('matched_size') * pl.col('imbalance_buy_sell_flag')).alias("div_flag_imbalance_size_2_balance"),
        ((pl.col('ask_price') - pl.col('bid_price')) * pl.col('imbalance_size')).alias('price_pressure'),
        ((pl.col('ask_price') - pl.col('bid_price')) * pl.col('imbalance_size') * pl.col('imbalance_buy_sell_flag')).alias('price_pressure_v2'),
        ((pl.col("ask_size") - pl.col("bid_size")) / (pl.col("far_price") - pl.col("near_price"))).alias("depth_pressure"),
        (pl.col("bid_size") / pl.col("ask_size")).alias("div_bid_size_ask_size"),
    ])
    feas_list.extend(['ask_money', 'bid_money', 'ask_auc_money','bid_auc_money',"ask_size_all","bid_size_all","volumn_size_all",
                        'volumn_money','volume_cont',"volumn_auc","volumn_auc_money","mid_price",
                        'mid_price_near_far','price_diff_ask_bid',"price_div_ask_bid","flag_imbalance_size","div_flag_imbalance_size_2_balance",
                        "price_pressure","price_pressure_v2","depth_pressure","flag_scale_imbalance_size","diff_ask_bid_size"]) 
     # 各种ratio
    # 提升微忽几微
    add_cols = []
    for col1, col2 in [
        ("imbalance_size","bid_size"),
        ("imbalance_size","ask_size"),
        ("matched_size","bid_size"),
        ("matched_size","ask_size"),
        ("imbalance_size","volume_cont"),
        ("matched_size","volume_cont"),
        ("auc_bid_size","bid_size"),
        ("auc_ask_size","ask_size"),
        ("bid_auc_money","bid_money"),
        ("ask_auc_money","ask_money"),
    ]:
        add_cols.append((pl.col(col1) / pl.col(col2)).alias(f"div_{col1}_2_{col2}"))
        feas_list.append(f"div_{col1}_2_{col2}")        
    df = df.with_columns(add_cols)
    # 阶段2 不平衡特征
    # 除了price相关
    # 没加auc的ask/bid的 构造price以及不平衡进去
    add_cols = []
    for pair1,pair2 in [
        ('ask_size','bid_size'),
        ('ask_money','bid_money'),
        ('volumn_money','volumn_auc_money'),
        ('volume_cont','volumn_auc'),
        ('imbalance_size','matched_size'),
        ('auc_ask_size','auc_bid_size'),
        ("ask_size_all",'bid_size_all')
    ]:
        col_imb = f"imb1_{pair1}_{pair2}"
        add_cols.extend([
            ((pl.col(pair1) - pl.col(pair2)) / (pl.col(pair1) + pl.col(pair2))).alias(col_imb),
        ])
        feas_list.extend([col_imb])
    df = df.with_columns(add_cols)


    # price侧的imb1
    fea_append_list = []
    prices = ["reference_price", "far_price", "near_price", "ask_price", "bid_price", "wap","mid_price"]
    for c in combinations(prices, 2):
        fea_append_list.append(((pl.col(c[0]) - pl.col(c[1])) / (pl.col(c[0]) + pl.col(c[1]))).alias(f"imb1_{c[0]}_{c[1]}"))
        # fea_append_list.append((pl.col(c[0]) - pl.col(c[1])).alias(f"diff_{c[0]}_{c[1]}"))
        feas_list.extend([f"imb1_{c[0]}_{c[1]}"])
    df = df.with_columns(fea_append_list)
    del fea_append_list
    # 不平衡特征 累计乘
    df = df.with_columns([
        ((pl.col("imb1_ask_size_bid_size") + 2) * (pl.col("imb1_ask_price_bid_price") + 2) * (pl.col("imb1_auc_ask_size_auc_bid_size")+2)).alias("market_urgency_v2"),
        (pl.col('price_diff_ask_bid') * (pl.col('imb1_ask_size_bid_size'))).alias('market_urgency'),
        (pl.col('imb1_ask_price_bid_price') * (pl.col('imb1_ask_size_bid_size'))).alias('market_urgency_v3'),
    ])
    feas_list.extend([f"market_urgency_v3",'market_urgency','market_urgency_v2'])

    feas_list = ['imb1_wap_mid_price', 'imb1_ask_money_bid_money', 'imb1_volume_cont_volumn_auc', 'imb1_reference_price_ask_price', 
                    'imb1_reference_price_mid_price', 'seconds_in_bucket', 'div_flag_imbalance_size_2_balance', 'ask_price', 
                    'imb1_reference_price_bid_price', 'scale_matched_size', 'imb1_near_price_wap', 'volumn_auc_money', 'imb1_far_price_wap', 
                    'bid_size', 'scale_bid_size', 'bid_size_all']
    # 隔离
    add_cols = []
    for col in ["bid_auc_money","imb1_reference_price_wap","bid_size_all",
                "imb1_auc_ask_size_auc_bid_size","div_flag_imbalance_size_2_balance",
                "imb1_ask_size_all_bid_size_all","flag_imbalance_size","imb1_reference_price_mid_price"]:
        for window in [3,6,18,36,60]:
            add_cols.append(pl.col(col).rolling_mean(window_size=window,min_periods=1).over('stock_id','date_id').alias(f'rolling{window}_mean_{col}'))
            add_cols.append(pl.col(col).rolling_std(window_size=window,min_periods=1).over('stock_id','date_id').alias(f'rolling{window}_std_{col}'))
            feas_list.extend([f'rolling{window}_mean_{col}',f'rolling{window}_std_{col}'])
    feas_list = ['imb1_wap_mid_price', 'imb1_ask_money_bid_money', 'imb1_volume_cont_volumn_auc', 
                        'imb1_reference_price_ask_price', 'imb1_reference_price_mid_price', 
                        'seconds_in_bucket', 'div_flag_imbalance_size_2_balance', 'ask_price', 
                        'imb1_reference_price_bid_price', 'scale_matched_size', 'imb1_near_price_wap', 
                        'volumn_auc_money', 'imb1_far_price_wap', 'bid_size', 'scale_bid_size', 'bid_size_all', 
                        'rolling18_mean_imb1_auc_ask_size_auc_bid_size', 'rolling3_mean_div_flag_imbalance_size_2_balance', 
                        'rolling60_std_div_flag_imbalance_size_2_balance', 'rolling36_mean_flag_imbalance_size', 
                        'rolling3_std_imb1_auc_ask_size_auc_bid_size', 'rolling18_mean_imb1_ask_size_all_bid_size_all', 
                        'rolling6_mean_div_flag_imbalance_size_2_balance', 'rolling6_std_imb1_auc_ask_size_auc_bid_size', 
                        'rolling3_mean_imb1_auc_ask_size_auc_bid_size', 'rolling60_std_imb1_auc_ask_size_auc_bid_size', 
                        'rolling6_std_bid_size_all', 'rolling3_std_bid_size_all', 'rolling3_mean_bid_size_all', 
                        'rolling18_std_bid_auc_money', 'rolling36_mean_bid_auc_money',"rolling60_mean_imb1_reference_price_wap",
                    'rolling18_mean_imb1_reference_price_wap', 'rolling3_mean_imb1_reference_price_mid_price']
    df = df.with_columns(add_cols)
    add_cols = []
    ### 杂七杂八
    df = df.with_columns([
        pl.col("flag_imbalance_size").diff().over('stock_id','date_id').alias("imbalance_momentum_unscaled"),
        pl.col("price_diff_ask_bid").diff().over('stock_id','date_id').alias("spread_intensity"),
    ])
    feas_list.extend(["imbalance_momentum_unscaled","spread_intensity"])
    df = df.with_columns([
        (pl.col("imbalance_momentum_unscaled")/pl.col("matched_size")).alias("imbalance_momentum")
    ])
    feas_list.extend(["imbalance_momentum"])

    #Calculate diff features for specific columns
    add_cols = []
    for col in ['ask_price',
        'bid_price',
        'imb1_reference_price_near_price',
        'bid_size',
        'scale_bid_size',
        'mid_price',
        'ask_size',
        'price_div_ask_bid',
        'div_bid_size_ask_size',
        'market_urgency',
        'wap',
        'imbalance_momentum']:
        for window in [1, 2, 3, 10]:
            add_cols.append((pl.col(col).diff(window).over('stock_id','date_id')).alias(f"{col}_diff_{window}"))
            feas_list.append(f"{col}_diff_{window}")
    df = df.with_columns(add_cols)

    ### target mock系列
    for mock_period in [1,3,12,6]:

        df = df.with_columns([
            pl.col("wap").shift(-mock_period).over("stock_id","date_id").alias(f"wap_shift_n{mock_period}")
        ])
        df = df.with_columns([
            (pl.col(f"wap_shift_n{mock_period}")/pl.col("wap")).alias("target_single")
        ])

        tmp_df = df.select(pl.col("target_single"),pl.col("weight")).to_pandas()
        tmp_df.loc[tmp_df["target_single"].isna(),"weight"] = 0
        df = df.with_columns([
            pl.lit(np.array(tmp_df["weight"])).alias("weight_tmp")
        ])

        df = df.with_columns([
            (((pl.col("weight_tmp") * pl.col("target_single")).sum().over("date_id","seconds_in_bucket")) / ((pl.col("weight_tmp")).sum().over("date_id","seconds_in_bucket"))).alias("index_target_mock")
        ])

        df = df.with_columns([
            ((pl.col("target_single") - pl.col("index_target_mock"))*10000).alias("target_mock")
        ])

        df = df.with_columns([
            pl.col("target_mock").shift(mock_period).over("stock_id","date_id").alias(f"target_mock_shift{mock_period}"),
        ])

    add_cols = []
    for col in ['target_mock_shift6','target_mock_shift1','target_mock_shift3','target_mock_shift12']:
        for window in [1, 3,6,12,24,48]:
            add_cols.append(pl.col(col).rolling_mean(window_size=window,min_periods=1).over('stock_id','date_id').alias(f'rolling{window}_mean_{col}'))
    df = df.with_columns(add_cols)
    keep_cols_new = ['rolling48_mean_target_mock_shift3', 'rolling48_mean_target_mock_shift1', 'rolling48_mean_target_mock_shift12',
    'rolling1_mean_target_mock_shift6', 'rolling24_mean_target_mock_shift6','rolling24_mean_target_mock_shift12',]
    feas_list.extend(keep_cols_new)

    add_cols = []
    for col in ["imb1_auc_ask_size_auc_bid_size","flag_imbalance_size","price_pressure_v2","scale_matched_size"]:
        for window_size in [1,2,3,6,12]:
            add_cols.append(pl.col(col).shift(window_size).over('stock_id','date_id').alias(f'shift{window_size}_{col}'))
            add_cols.append((pl.col(col) / pl.col(col).shift(window_size).over('stock_id','date_id')).alias(f'div_shift{window_size}_{col}'))
            add_cols.append((pl.col(col) - pl.col(col).shift(window_size).over('stock_id','date_id')).alias(f'diff_shift{window_size}_{col}'))
    feas_list.extend(['div_shift6_imb1_auc_ask_size_auc_bid_size',
    'diff_shift6_price_pressure_v2',
    'shift1_price_pressure_v2',
    'div_shift3_flag_imbalance_size',
    'div_shift12_imb1_auc_ask_size_auc_bid_size',
    'div_shift3_scale_matched_size',
    'diff_shift6_flag_imbalance_size',
    'shift12_imb1_auc_ask_size_auc_bid_size',
    'div_shift12_price_pressure_v2',
    'shift6_flag_imbalance_size',
    'diff_shift3_imb1_auc_ask_size_auc_bid_size',
    'div_shift12_flag_imbalance_size',
    'shift12_flag_imbalance_size'])
    df = df.with_columns(add_cols)

    add_cols = []
    for col in ['imb1_ask_price_mid_price',
    'market_urgency',
    'market_urgency_diff_1',
    'imb1_ask_money_bid_money',
    'rolling18_mean_imb1_ask_size_all_bid_size_all',
    'rolling18_mean_imb1_auc_ask_size_auc_bid_size',
    'rolling18_mean_imb1_reference_price_wap',
    'ask_price_diff_3',
    'diff_shift1_price_pressure_v2',
    'diff_shift12_scale_matched_size',
    'diff_shift1_flag_imbalance_size',
    'imb1_ask_size_bid_size',
    'imb1_bid_price_mid_price',
    'rolling48_mean_target_mock_shift6']:
        add_cols.append((((pl.col(col) * pl.col("weight")).sum().over("date_id","seconds_in_bucket"))/(((pl.col("weight")).sum().over("date_id","seconds_in_bucket")))).alias(f"global_{col}"))
        feas_list.append(f"global_{col}")
    df = df.with_columns(add_cols)


    # MACD
    rsi_cols = ["mid_price_near_far","imb1_reference_price_wap","near_price",]
    add_cols = []
    for col in rsi_cols:
        for window_size in [3,6,12,24,48]:
            add_cols.append(pl.col(col).ewm_mean(span=window_size, adjust=False).over('stock_id','date_id').alias(f"rolling_ewm_{window_size}_{col}"))
    df = df.with_columns(add_cols)

    add_cols = []
    for col in rsi_cols:
        for w1,w2 in zip((3,6,12,24),(6,12,24,48)):
            add_cols.append((pl.col(f"rolling_ewm_{w1}_{col}") - pl.col(f"rolling_ewm_{w2}_{col}")).alias(f"dif_{col}_{w1}_{w2}"))
    df = df.with_columns(add_cols)

    add_cols = []
    for col in rsi_cols:
        for w1,w2 in zip((3,6,12,24),(6,12,24,48)):
            add_cols.append(pl.col(f"dif_{col}_{w1}_{w2}").ewm_mean(span=9, adjust=False).over('stock_id','date_id').alias(f"dea_{col}_{w1}_{w2}"))
    df = df.with_columns(add_cols)

    add_cols = []
    for col in rsi_cols:
        for w1,w2 in zip((3,6,12,24),(6,12,24,48)):
            add_cols.append((pl.col(f"dif_{col}_{w1}_{w2}") - pl.col(f"dea_{col}_{w1}_{w2}")).alias(f"macd_{col}_{w1}_{w2}"))

    feas_list.extend(['macd_imb1_reference_price_wap_12_24',
    'dif_imb1_reference_price_wap_3_6',
    'macd_mid_price_near_far_12_24',
    'dif_near_price_3_6',
    'macd_near_price_24_48',
    'dea_imb1_reference_price_wap_12_24',
    'macd_near_price_12_24',
    'rolling_ewm_24_imb1_reference_price_wap',
    'dif_near_price_6_12',
    'dea_mid_price_near_far_6_12',
    'dea_near_price_24_48',
    'rolling_ewm_12_imb1_reference_price_wap',
    'dif_imb1_reference_price_wap_12_24'])
    df = df.with_columns(add_cols)
    # MACD
    rsi_cols = ["mid_price_near_far","imb1_reference_price_wap","near_price",]
    add_cols = []
    for col in rsi_cols:
        for window_size in [3,6,12,24,48]:
            add_cols.append(pl.col(col).ewm_mean(span=window_size, adjust=False).over('stock_id','date_id').alias(f"rolling_ewm_{window_size}_{col}"))
            #feas_list.append(f"rolling_ewm_{window_size}_{col}")
    df = df.with_columns(add_cols)

    add_cols = []
    for col in rsi_cols:
        for w1,w2 in zip((3,6,12,24),(6,12,24,48)):
            add_cols.append((pl.col(f"rolling_ewm_{w1}_{col}") - pl.col(f"rolling_ewm_{w2}_{col}")).alias(f"dif_{col}_{w1}_{w2}"))
            #feas_list.append(f"dif_{col}_{w1}_{w2}")
    df = df.with_columns(add_cols)

    add_cols = []
    for col in rsi_cols:
        for w1,w2 in zip((3,6,12,24),(6,12,24,48)):
            add_cols.append(pl.col(f"dif_{col}_{w1}_{w2}").ewm_mean(span=9, adjust=False).over('stock_id','date_id').alias(f"dea_{col}_{w1}_{w2}"))
            #feas_list.append(f"dea_{col}_{w1}_{w2}")
    df = df.with_columns(add_cols)

    add_cols = []
    for col in rsi_cols:
        for w1,w2 in zip((3,6,12,24),(6,12,24,48)):
            add_cols.append((pl.col(f"dif_{col}_{w1}_{w2}") - pl.col(f"dea_{col}_{w1}_{w2}")).alias(f"macd_{col}_{w1}_{w2}"))
            #feas_list.append(f"macd_{col}_{w1}_{w2}")

    feas_list.extend(['macd_imb1_reference_price_wap_12_24',
    'dif_imb1_reference_price_wap_3_6',
    'macd_mid_price_near_far_12_24',
    'dif_near_price_3_6',
    'macd_near_price_24_48',
    'dea_imb1_reference_price_wap_12_24',
    'macd_near_price_12_24',
    'rolling_ewm_24_imb1_reference_price_wap',
    'dif_near_price_6_12',
    'dea_mid_price_near_far_6_12',
    'dea_near_price_24_48',
    'rolling_ewm_12_imb1_reference_price_wap',
    'dif_imb1_reference_price_wap_12_24'])
    df = df.with_columns(add_cols)
    del add_cols
    tot_feas = df.join(df_pl,how='left',on=['stock_id','date_id','seconds_in_bucket'])
    del df
    return tot_feas
    """
rank9_features = rank9_features_from_pic(ORIGINAL_TRAIN.copy())

Function 'rank9_features_from_pic' executed in 27.1509 seconds


In [7]:
rank9_features.write_parquet("rank9_features.parquet")

# 4.ipynb中的因子——wyn

In [5]:
@njit(parallel=True)
def compute_triplet_imbalance(df_values, comb_indices):
    num_rows = df_values.shape[0]
    num_combinations = len(comb_indices)
    imbalance_features = np.empty((num_rows, num_combinations))

    for i in prange(num_combinations):
        a, b, c = comb_indices[i]
        for j in range(num_rows):
            max_val = max(df_values[j, a], df_values[j, b], df_values[j, c])
            min_val = min(df_values[j, a], df_values[j, b], df_values[j, c])
            mid_val = df_values[j, a] + df_values[j, b] + df_values[j, c] - min_val - max_val
            if mid_val == min_val:  # Prevent division by zero
                imbalance_features[j, i] = np.nan
            else:
                imbalance_features[j, i] = (max_val - mid_val) / (mid_val - min_val)

    return imbalance_features
def calculate_triplet_imbalance_numba(price, df):
    # Convert DataFrame to numpy array for Numba compatibility
    df_values = df[price].values
    comb_indices = [(price.index(a), price.index(b), price.index(c)) for a, b, c in combinations(price, 3)]

    # Calculate the triplet imbalance
    features_array = compute_triplet_imbalance(df_values, comb_indices)

    # Create a DataFrame from the results
    columns = [f"{a}_{b}_{c}_imb2" for a, b, c in combinations(price, 3)]
    features = pd.DataFrame(features_array, columns=columns)

    return features

In [6]:
@time_it
def features_from_ipynb(df):
    def generate_features(df):
    
        prices = ['reference_price', 'far_price', 'mid_price',
                  'near_price', 'ask_price', 'bid_price', 'wap']
        sizes = ["matched_size", "bid_size", "ask_size", "imbalance_size"]

        df['mid_price'] = (df['ask_price']+df['bid_price'])/2

        for i, a in enumerate(prices):
            for j, b in enumerate(prices):
                if i > j:
                    df[f'{a}_{b}_imb'] = (df[a]-df[b])/(df[a]+df[b])

        for c in [prices, sizes]:
            triplet_feature = calculate_triplet_imbalance_numba(c, df)
            df[triplet_feature.columns] = triplet_feature.values

        df['spread'] = df['ask_price']-df['bid_price']
        df['price_pressure'] = df['imbalance_size']*df['spread']
        df['ask_amount'] = df['ask_price']*df['ask_size']
        df['bid_amount'] = df['bid_price']*df['bid_size']
        df['marketdepth'] = df['ask_size']+df['bid_size']
        df['liquidity_imbalance'] = (
            df['bid_size']-df['ask_size'])/df['marketdepth']
        df['matched_imbalance'] = (df['imbalance_size']-df['matched_size']) / \
            (df['imbalance_size']+df['matched_size'])
        df['wap_2'] = (df['ask_amount']+df['bid_amount'])/df['marketdepth']
        df['bboimb'] = (df['bid_amount']-df['ask_amount']) / \
            (df['ask_amount']+df['bid_amount'])
        df['price_imbalance'] = df['bid_price']/df['ask_price']
        df['size_imbalance'] = df['bid_size']/df['ask_size']
        df['market_urgency'] = df['spread'] * df['liquidity_imbalance']
        df['effectiveSpread'] = np.abs(df['wap']-df['mid_price'])/df['mid_price']
        df['shallowLIX'] = np.log(
            df['matched_size']/(df['ask_price']-df['bid_price']))
        df['imbalance_ratio'] = df['imbalance_size'] / \
            (df['imbalance_size']+df['matched_size'])
        df['logquoteslope'] = (np.log(df['ask_price'])-np.log(df['bid_price'])) / \
            (np.log(df['ask_size'])+np.log(df['bid_size']))
        df['wapm'] = (df['wap_2']-df['mid_price'])/df['mid_price']
        df['MCI'] = df['wapm']/(df['ask_amount']+df['bid_amount'])
        df['imb_size_with_flag'] = (2 * df['imbalance_size'] *
                                    df['imbalance_buy_sell_flag'])/df['marketdepth']
        df['farrefliquidity'] = (
            df['far_price']/df['reference_price']-1)/df['matched_size']
        df['farmidliquidity'] = (
            df['far_price']/df['mid_price']-1)/df['matched_size']
        df['farnearliquidity'] = (
            df['far_price']/df['near_price']-1)/df['matched_size']
        df['farwapliquidity'] = (df['far_price']/df['wap']-1)/df['matched_size']
        df['depth_pressure'] = (df['ask_size']-df['bid_size']) * \
            (df['far_price']-df['near_price'])
        return df
    
    def cal_extra_fea(df_original_fea,
                  window=[6, 12],
                  diff_window=[1, 2, 3, 5, 10],
                  fac_to_remain=None):
        '''
        df_original_fea: pd.Series, index为datetime和security, name为因子名称
        window: list
        fac_to_remain: list, 指保留的因子名称
        '''
        f = df_original_fea.name
        fea_wide = df_original_fea.unstack().T

        feature = pd.DataFrame(index=df_original_fea.index)

        # 预先计算 rolling 对象以减少重复计算
        rolling_windows = {w: fea_wide.rolling(w, min_periods=1) for w in window}

        for w in diff_window:
            # 计算差分 (diff)
            fname = f'{f}_diff{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = (fea_wide - fea_wide.shift(w)).stack(dropna=False)

            # 计算动量 (momentum)
            fname = f'{f}_mom{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = (fea_wide / fea_wide.shift(w) - 1).stack(dropna=False)

        for w, roll in rolling_windows.items():
            # 滚动均值 (ma)
            fname = f'{f}_ma{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.mean().stack(dropna=False)

            # 80%分位数 (qtlu)
            fname = f'{f}_qtlu{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.quantile(0.8).stack(dropna=False)

            # 20%分位数 (qtld)
            fname = f'{f}_qtld{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.quantile(0.2).stack(dropna=False)

            # 排名百分比 (rank)
            fname = f'{f}_rank{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.rank(pct=True).stack(dropna=False)

            # 标准差 (std)
            fname = f'{f}_std{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.std().stack(dropna=False)

            # 最大值 (max)
            fname = f'{f}_max{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.max().stack(dropna=False)

            # 最小值 (low)
            fname = f'{f}_low{w}'
            if fac_to_remain is None or fname in fac_to_remain:
                feature[fname] = roll.min().stack(dropna=False)
        return feature

#             # 计算最大值的相对位置 (imax)
#             fname = f'{f}_imax{w}'

#             def findMaxIdx(series): 
#                 return series.shape[0] - series.reset_index(drop=True).idxmax()

#             if fac_to_remain is None or fname in fac_to_remain:
#                 feature[fname] = roll.apply(findMaxIdx).stack(dropna=False) / w

#             # 计算最小值的相对位置 (imin) 
#             fname = f'{f}_imin{w}'

#             def findMinIdx(series): 
#                 return series.shape[0] - series.reset_index(drop=True).idxmin()

#             if fac_to_remain is None or fname in fac_to_remain:
#                 feature[fname] = roll.apply(findMinIdx).stack(dropna=False) / w
            
    df1 = generate_features(df.copy())
    result = df1
    print('---------df1 done-----------')
    """
    df2 = df.copy()
    df2.set_index(['stock_id', 'time_id'], inplace=True)
    for f in ['imbalance_size','reference_price','matched_size','near_price','bid_price','bid_size','ask_price','ask_size','wap']:
        extra_fea = cal_extra_fea(df2[f],window=[6, 12],diff_window=[1, 5, 10],fac_to_remain=None)
        extra_fea = extra_fea.reset_index()
        print(result.shape,extra_fea.shape)
        result = pd.merge(result,extra_fea,how='left',on=['stock_id','time_id'])
    """
    return result
ipynb_features = features_from_ipynb(ORIGINAL_TRAIN.copy())

---------df1 done-----------
Function 'features_from_ipynb' executed in 12.4721 seconds


In [8]:
ipynb_features = pl.DataFrame(ipynb_features)